In [1]:
# !pip install pulp

In [2]:
import pandas as pd
import numpy as np
import pulp
import warnings
warnings.filterwarnings('ignore')
from pulp import *
prob = LpProblem("Matching Employees", LpMaximize)

In [3]:
np.random.seed(0)

c = np.random.randint(0,10, (8,8))
np.fill_diagonal(c,0)
c

array([[0, 0, 3, 3, 7, 9, 3, 5],
       [2, 0, 7, 6, 8, 8, 1, 6],
       [7, 7, 0, 1, 5, 9, 8, 9],
       [4, 3, 0, 0, 5, 0, 2, 3],
       [8, 1, 3, 3, 0, 7, 0, 1],
       [9, 9, 0, 4, 7, 0, 2, 7],
       [2, 0, 0, 4, 5, 5, 0, 8],
       [4, 1, 4, 9, 8, 1, 1, 0]])

In [4]:
employees = range(8)
group = 4

In [5]:
#Define the variables
y = LpVariable.dicts("pair", [(i,j) for i in employees for j in employees] ,cat='Binary')

In [6]:
names = ['Ben','Kate','Thinh','Jorge','Alfredo','Francisco','Olivia','Chris']
match_info = pd.DataFrame(c, index=names, columns=names)
match_info

,Ben,Kate,Thinh,Jorge,Alfredo,Francisco,Olivia,Chris
Ben,0,0,3,3,7,9,3,5
Kate,2,0,7,6,8,8,1,6
Thinh,7,7,0,1,5,9,8,9
Jorge,4,3,0,0,5,0,2,3
Alfredo,8,1,3,3,0,7,0,1
Francisco,9,9,0,4,7,0,2,7
Olivia,2,0,0,4,5,5,0,8
Chris,4,1,4,9,8,1,1,0


In [7]:
prob += lpSum([(c[i][j] + c[j][i]) * y[(i,j)] for i in employees for j in employees])

In [8]:
for i in employees:
    prob += lpSum(y[(i,j)] for j in employees) <= 1
    prob += lpSum(y[(j,i)] for j in employees) <= 1
    prob += lpSum(y[(i,j)] for j in employees)+ lpSum(y[(j,i)] for j in employees) <= 1
prob += lpSum(y[(i,j)] for i in employees for j in employees) == 4
prob.solve()

In [10]:
print("Finish matching!\n")
for i in employees:
    for j in employees:
        if y[(i,j)].varValue == 1:
            print('{} and {} with preference score {} and {}. Total score: {}'.format(names[i],
                                                                                      names[j],c[i,j],
                                                                                      c[j,i],
                                                                                      c[i,j] +c[j,i]))

Finish matching!

Ben and Alfredo with preference score 7 and 8. Total score: 15
Kate and Francisco with preference score 8 and 9. Total score: 17
Jorge and Chris with preference score 3 and 9. Total score: 12
Olivia and Thinh with preference score 0 and 8. Total score: 8


In [11]:
pulp.value(prob.objective)

52.0

In [12]:
pulp.LpStatus[prob.status]

'Optimal'